In [ ]:
pip install --upgrade ultralytics
!pip install yolov5


In [6]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
import random
from tracker import Tracker  # Assuming you have a Tracker class for DeepSORT

In [2]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)

In [7]:



cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)
# Define video paths and output filename
video_path = "D:/Project_data/Test_vid.mp4"
video_out_path = os.path.join('.', 'out.mp4')

# Define areas for entering and leaving
area1 = [(312, 388), (289, 390), (474, 469), (497, 462)]
area2 = [(279, 392), (250, 397), (423, 477), (454, 469)]

# Load YOLOv8 model
model = YOLO("yolov8s.pt")

# Initialize tracker
tracker = Tracker()

# Generate random colors for bounding boxes
colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for j in range(10)]

# Minimum detection confidence threshold
detection_threshold = 0.5

# Open video capture
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video source.")
    exit()

# Open video writer for output
cap_out = cv2.VideoWriter(video_out_path, cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS),
                          (frame.shape[1], frame.shape[0]))

# Data structures for people entering/leaving
people_entering = {}
people_leaving = {}
entering = set()
leaving = set()

while ret:

    # Make video processing efficient (process every other frame)
    if not ret or cv2.waitKey(1) & 0xFF == 27:
        break

    # Read frame
    ret, frame = cap.read()

    # Detect objects using YOLOv8
    results = model(frame)

    # Extract relevant information from detections
    detections = []
    for result in results:
        for r in result.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = r
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)
            class_id = int(class_id)
            if score > detection_threshold and 'person' in class_list[class_id]:  # Filter for 'person' class
                detections.append([x1, y1, x2, y2])

    # Update tracker with detections
    tracker.update(frame, detections)

    # Process tracked objects
    for track in tracker.tracks:
        bbox = track.bbox
        x1, y1, x2, y2 = bbox  
        track_id = track.track_id

        # Check for entering zone
        results = cv2.pointPolygonTest(np.array(area2, np.int32), ((x2, y2)), False)  
        if results >= 0 and track_id not in people_entering:
            people_entering[track_id] = (x2, y2)
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0, 0, 255), 2)
        if track_id in people_entering:
            results1=cv2.pointPolygonTest(np.array(area1,np.int32),((x2,y2)),False)
            if results1>=0:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.circle(frame,(x2,y2),5,(255,0,255),-1)
                cv2.putText(frame, str(track_id), (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                entering.add(track_id)

        # Check for leaving zone (if object previously entered)
        results2=cv2.pointPolygonTest(np.array(area1,np.int32),((x2,y2)),False)
        if results2>=0:
            people_leaving[track_id]=(x2,y2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        if track_id in people_leaving:
            results3=cv2.pointPolygonTest(np.array(area2,np.int32),((x2,y2)),False)
            if results3>=0:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255,0 , 255), 2)
                cv2.circle(frame,(x2,y2),5,(255,0,255),-1)
                cv2.putText(frame, str(track_id), (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                leaving.add(track_id)

                
    cv2.polylines(frame,[np.array(area1,np.int32)],True,(255,0,0),2)
    cv2.putText(frame,str('1'),(504,471),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)

    cv2.polylines(frame,[np.array(area2,np.int32)],True,(255,0,0),2)
    cv2.putText(frame,str('2'),(466,485),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)
    
    cv2.putText(frame,str(len(entering)-len(leaving)),(60,80),cv2.FONT_HERSHEY_COMPLEX,(0.7),(0,0,255),2)
    cv2.imshow("RGB", frame)
    

cap.release()
cap_out.release()
cv2.destroyAllWindows()


0: 384x640 18 persons, 4 cars, 1 motorcycle, 6 backpacks, 1 handbag, 451.7ms
Speed: 8.2ms preprocess, 451.7ms inference, 1330.6ms postprocess per image at shape (1, 3, 384, 640)


NameError: name 'class_list' is not defined